## 图像几何变换

仿射变换和透视变换

参考：

[官方文档](https://docs.opencv.org/4.1.2/da/d6e/tutorial_py_geometric_transformations.html)

[仿射变换及其变换矩阵的理解](https://www.cnblogs.com/shine-lee/p/10950963.html)

[Perspective Transformation](https://theailearner.com/2020/11/06/perspective-transformation/)

[OpenCv Perspective Transformation](https://medium.com/analytics-vidhya/opencv-perspective-transformation-9edffefb2143)

## 仿射变换(Affine)

仿射变换包括如下所有变换，以及这些变换任意次序次数的组合:

![仿射变换](./images/仿射变换.png)

![仿射变换Venn图](./images/仿射变换Venn图.png)

- 平移和旋转的组合是欧式变换或刚体变换
- 放缩可进一步分为uniform scaling和non-uniform scaling，前者每个坐标轴放缩系数相同（各向同性），后者不同；如果放缩系数为负，则会叠加上反射，可以看成是特殊的scaling
- 刚体变换+uniform scaling 称之为，相似变换（similarity transformation），即平移+旋转+各向同性的放缩
- 剪切变换（shear mapping）将所有点沿某一指定方向成比例地平移
- 排除了平移变换的所有仿射变换为线性变换

使用齐次变换矩阵可以表示以上所有变换形式:

$$\begin{bmatrix} x' \\ y' \\ 1 \end{bmatrix} = \begin{bmatrix} a & b & c \\ d & e & f \\ 0 & 0 & 1 \end{bmatrix} \begin{bmatrix} x \\ y \\ 1 \end{bmatrix}$$

不同的基础变换对应的a,b,c,d,e,f约束不同：

![仿射变换矩阵](./images/仿射变换矩阵.png)

上图中的旋转矩阵是顺时针旋转的，如果是逆时针旋转，则将$sin$项取负即可

### 变换矩阵理解与记忆

当未旋转时，假设有坐标系中一点$(x,y)$，其相对坐标原点在$[1,0]$方向上的投影为$x$，在$[0,1]$方向上的投影为$y$，则：

$$\begin{bmatrix}x\\y\end{bmatrix}=x\begin{bmatrix}1\\0\end{bmatrix}+y\begin{bmatrix}0\\1\end{bmatrix}=\begin{bmatrix}1&0\\0&1\end{bmatrix}\begin{bmatrix}x\\y\end{bmatrix}$$

当坐标系旋转时，坐标系中的点也随之旋转，但点相对于坐标系的位置不变，仍为$(x,y)$，当逆时针旋转$\theta$角后，新坐标轴基向量变为$\begin{bmatrix}\cos(\theta)&\sin(\theta)\end{bmatrix}$，$\begin{bmatrix}-\sin(\theta)&\cos(\theta)\end{bmatrix}$，点的坐标表示为：

$$\begin{bmatrix}x' \\y'\end{bmatrix}= x\begin{bmatrix}\cos(\theta) \\\sin(\theta)\end{bmatrix}+ y\begin{bmatrix}-\sin(\theta) \\\cos(\theta)\end{bmatrix}= \begin{bmatrix}\cos(\theta) & -\sin(\theta) \\
\sin(\theta) & \cos(\theta)\end{bmatrix}\begin{bmatrix}x \\y\end{bmatrix}$$

其他变换矩阵也是同理：所有变换矩阵只需关注一点：坐标系的变化，即基向量和原点的变化；坐标系变化到哪里，坐标系中的所有点也跟着做同样的变化。

在仿射变换矩阵$\begin{bmatrix} a & b & c \\ d & e & f \\ 0 & 0 & 1 \end{bmatrix}$中，$\begin{bmatrix}a\\d\end{bmatrix}$和$\begin{bmatrix}b\\e\end{bmatrix}$为新的基向量，$\begin{bmatrix}c\\f\end{bmatrix}$为新的坐标原点，先变化基向量，再变化坐标原点

### 函数

opencv提供两个函数进行转换：

- `cv2.warpAffine` 采用2x3变换矩阵
- `cv2.warpPerspective` 采用3x3变换矩阵，一般用于透视变换

函数原型：

`cv2.warpAffine(src, M, dsize, dst=None, flags=None, borderMode=None, borderValue=None)`

参数说明：

- `src`：输入图像
- `M`：2x3仿射变换矩阵
- `dsize`：输出图像的大小，格式为(width, height)，宽度=列数，高度=行数
- `dst`：输出图像，如果提供，则函数将结果写入这个图像，而不是创建一个新的图像。同时，输出图像的大小必须与dsize匹配，类型与输入图像相同。在性能要求高的应用（如实时视频处理、批量图像处理）中，合理使用dst参数可以显著提升效率
  > dst = np.zeros_like(src)  # 只分配一次内存
  >
  > cv2.resize(src, (800, 600), dst)
  >
  > cv2.GaussianBlur(dst, (5, 5), 0, dst)  # 注意：这里dst既是输入也是输出
- `flags`：插值方法，默认值为`cv2.INTER_LINEAR`，可选值包括：
  - `cv2.INTER_NEAREST`：最近邻插值
  - `cv2.INTER_LINEAR`：双线性插值
  - `cv2.INTER_CUBIC`：4x4像素邻域的双三次插值
  - `cv2.INTER_LANCZOS4`：8x8像素邻域的Lanczos插值
- `borderMode`：边界模式，默认值为`cv2.BORDER_CONSTANT`，可选值包括：
  - `cv2.BORDER_CONSTANT`：常数边界
  - `cv2.BORDER_REPLICATE`：复制边界
  - `cv2.BORDER_REFLECT`：反射边界
  - `cv2.BORDER_WRAP`：环绕边界
- `borderValue`：边界值，当`borderMode`为`cv2.BORDER_CONSTANT`时使用，默认值为0

返回值：

- 输出图像

函数原型：

`cv2.warpPerspective(src, M, dsize, dst=None, flags=None, borderMode=None, borderValue=None)`

参数说明：

- `src`：输入图像
- `M`：3x3透视变换矩阵
- `dsize`：输出图像的大小，格式为(width, height)
- `dst`：输出图像，和输入图像大小相同
- `flags`：插值方法，默认值为`cv2.INTER_LINEAR`，可选值同上
- `borderMode`：边界模式，默认值为`cv2.BORDER_CONSTANT`，可选值同上
- `borderValue`：边界值，当`borderMode`为`cv2.BORDER_CONSTANT`时使用，默认值为0

返回值：

- 输出图像

In [2]:
import cv2
import numpy as np
from my_function import imshow

### 平移

将图像中所有的点按照指定的平移量水平或者垂直移动

In [3]:
img=cv2.imread("./images/lena.png")

M=np.float32([[1,0,50],[0,1,50]]) # 注意数组类型
translation=cv2.warpAffine(img,M,(300,300))

imshow(trans=translation)

### 旋转

将图像相对于某点旋转$\theta$角

建议使用`cv2.getRotationMatrix2D`生成旋转矩阵

函数原型：

`cv2.getRotationMatrix2D(center, angle, scale)`

参数说明：

- center: 旋转中心点，格式为 (x, y) 坐标元组
- angle: 旋转角度，单位为度
- 正数：逆时针旋转
- 负数：顺时针旋转
- scale: 各向同性缩放因子
  - 1.0: 保持原尺寸
  - 大于1.0: 放大
  - 小于1.0: 缩小

返回值：

- 返回一个 2×3 的仿射变换矩阵

In [7]:
std_M=cv2.getRotationMatrix2D((128,128),-45,1.0)
print(std_M)
rotation=cv2.warpAffine(img,std_M,(256,256))

imshow(rotation=rotation)

[[  0.70710678  -0.70710678 128.        ]
 [  0.70710678   0.70710678 -53.01933598]]


注意，在无平移向量的情况下，图像的旋转是绕原点进行的，而图像的原点是左上角角点

### 缩放

图像缩放是指图像大小按指定的比例进行放大或缩小

- isotropic scaling: 各向同性缩放，即每个坐标轴的缩放系数相同
- reflection: 反射，即关于某一轴对称，也是一种缩放

缩小图像称为下采样(subsampled)或downsampling，放大图像称为上采样(upsampled)(upsampling)，主要目的是得到更高分辨率的图像

两种方法实现缩放：

- 变换矩阵+warpAffine函数
- 直接使用`cv2.resize`函数

`dst=cv2.resize(src, dsize, dst, fx, fy, interpolation)`

参数说明：

- `src`：输入图像
- `dsize`：输出图像的大小，格式为(width, height)
- `dst`：输出图像，和输入图像大小相同
- `fx`：水平方向的缩放因子
- `fy`：垂直方向的缩放因子
- `interpolation`：插值方法，默认值为`cv2.INTER_LINEAR`，可选值同上

返回值：

- 输出图像

In [12]:
M=np.float32([[2,0,0],[0,4,0]])
scale=cv2.warpAffine(img,M,(512,1024))

scale_resize=cv2.resize(img,(512,1024),dst=None,fx=2,fy=4)

imshow(scale=scale,scale_resize=scale_resize)

### 求解仿射变换矩阵

使用函数`cv2.getAffineTransform(srcPoints, dstPoints)`

参数说明：

- `srcPoints`：输入图像中的三个点的坐标，格式为numpy数组，形状为(3, 2)
- `dstPoints`：输出图像中的三个点的坐标，格式为numpy数组，形状为(3, 2)

返回值：

- 返回一个 2×3 的仿射变换矩阵

### 仿射变换的应用

神经网络的正向传播中进行的矩阵的乘积运算和加法运算（w,b）本质上也是一种仿射变换，因而，我们一般称神经网络中的线性层为仿射层（affine layer）

通过仿射变换对图像进行旋转、平移、缩放等操作以达到数据增强的效果，在训练深度学习模型时非常常见

## 透视变换(Perspective)

- 透视变换是一种非线性变换，它可以通过调整图像中物体的尺寸和位置关系，使其看起来更符合人眼的视觉感受，这也是其名字“透视”的由来。透视变换需要至少4个点来确定一个透视变换矩阵；
- 仿射变换是一种线性变换，它可以通过调整图像中物体的位置、旋转和缩放关系，来实现图像的变换。仿射变换需要至少3个点来确定一个仿射变换矩阵。

因此，透视变换和仿射变换虽然都可以实现图像的变换，但它们的应用场景和变换效果是不同的。透视变换主要用于处理三维场景中的透视投影问题，而仿射变换则更适用于处理二维场景中的平移、旋转和缩放等问题。

在仿射变换中，齐次变换矩阵为：

$$\begin{bmatrix} a & b & c \\ d & e & f \\ 0 & 0 & 1 \end{bmatrix}$$

左下角的$1\times2$的零向量称为投影向量，在仿射变换中投影向量为零向量，而在透视变换中投影向量不为零向量，从而引入了透视畸变的效果。故透视变换矩阵为：

$$\begin{bmatrix} a & b & c \\ d & e & f \\ g & h & 1 \end{bmatrix}$$

由于变换矩阵有8个自由度，因此需要至少4个点来确定一个透视变换矩阵，根据用途将这4个点映射到未知输出图像中的所需位置，从而计算出变换矩阵

![透视变换](./images/透视变换.png)

### opencv实现

函数原型：

`cv2.getPerspectiveTransform(srcPoints, dstPoints)`

参数说明：

- `srcPoints`：输入图像中的四个点的坐标，格式为numpy数组，形状为(4, 2)
- `dstPoints`：输出图像中的四个点的坐标，格式为numpy数组，形状为(4, 2)

返回值：
- 返回一个 3×3 的透视变换矩阵

现在读取一张书本图像，结合之前所学的轮廓检测技术，检测出书本的四个角点，然后通过透视变换将书本矫正为正面朝向的矩形图像

In [ ]:
book=cv2.imread("./images/book.jpg")
gray_book=cv2.cvtColor(book,cv2.COLOR_BGR2GRAY)
_,binary_book=cv2.threshold(gray_book,166,255,cv2.THRESH_BINARY)
edge_book=cv2.morphologyEx(binary_book,cv2.MORPH_GRADIENT,np.ones((5,5),np.uint8))
imshow(edge_book=edge_book)
contours,_=cv2.findContours(edge_book, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
book_contour = None
max_area = 0
for contour in contours:
    # 计算轮廓面积
    area = cv2.contourArea(contour)
    # 过滤掉太小的轮廓
    if area < 1000:
        continue
    # 多边形近似
    epsilon=0.02*cv2.arcLength(contour, True)
    approx=cv2.approxPolyDP(contour, epsilon, True)
    # 寻找四边形（书本通常是四边形）
    if len(approx) == 4:
        # 选择面积最大的四边形
        if area > max_area:
            max_area = area
            book_contour = approx
# 输出结果
if book_contour is None:
    print("未找到书本轮廓")
else:
    print("找到书本轮廓")
    cv2.drawContours(book, [book_contour], -1, (0, 0, 255), 3)
    imshow(book_with_contour=book)

找到书本轮廓


In [59]:
src_point = np.float32([book_contour[1][0],book_contour[0][0] , book_contour[3][0], book_contour[2][0]])
dst_point = np.float32([[0, 0], [450, 0], [450, 650], [0, 650]])
M = cv2.getPerspectiveTransform(src_point, dst_point)
warped_book = cv2.warpPerspective(book, M, (450,650))
imshow(warped_book=warped_book,book=book)

读取并转化后的图像不太理想，想要更精准，最好手动选点